In [4]:
import json
from tqdm import tqdm

In [2]:
with open('../language-detection/dumping-twitter-6-july-2019.json') as fopen:
    twitter = json.load(fopen)
    
len(twitter)

6597867

In [5]:
files = ['2020-02-22-twitter-dump-in.json', '2020-03-08-twitter-dump-in.json']

for file in files:
    file = f'../language-detection/{file}'
    print(file)

    with open(file) as fopen:
        temp = json.load(fopen)

    for i in tqdm(range(len(temp))):
        retweet = temp[i]['retweet_text_full']
        t = temp[i]['data_text']
        if retweet != 'NULL' and len(retweet) > len(t):
            t = retweet
        twitter.append(t)

../language-detection/2020-02-22-twitter-dump-in.json


100%|██████████| 1617795/1617795 [00:01<00:00, 970771.39it/s] 


../language-detection/2020-03-08-twitter-dump-in.json


100%|██████████| 1711555/1711555 [00:01<00:00, 967862.72it/s]


In [8]:
len(twitter)

9927217

In [7]:
import cleaning

In [9]:
import re

def preprocessing(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(
        u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]',
        ' ',
        string,
        flags = re.UNICODE,
    )
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

In [10]:
cleaned = []

for i in tqdm(range(len(twitter))):
    c = preprocessing(twitter[i])
    cleaned.append((twitter[i], c))

100%|██████████| 9927217/9927217 [03:27<00:00, 47956.30it/s]


In [59]:
negate_words = {
    'tak',
    'jangan',
    'tidak',
    'enggak',
    'tiada',
    'bukan',
    'usah',
    'tidaklah',
    'jgn',
    'tk',
    'bkn',
    "shouldnt",
    "dont",
    "doesnt",
}

import itertools

def _pad_sequence(
    sequence,
    n,
    pad_left = False,
    pad_right = False,
    left_pad_symbol = None,
    right_pad_symbol = None,
):
    sequence = iter(sequence)
    if pad_left:
        sequence = itertools.chain((left_pad_symbol,) * (n - 1), sequence)
    if pad_right:
        sequence = itertools.chain(sequence, (right_pad_symbol,) * (n - 1))
    return sequence

def ngrams(
    sequence,
    n: int,
    pad_left = False,
    pad_right = False,
    left_pad_symbol = None,
    right_pad_symbol = None,
):
    sequence = _pad_sequence(
        sequence, n, pad_left, pad_right, left_pad_symbol, right_pad_symbol
    )

    history = []
    while n > 1:
        try:
            next_item = next(sequence)
        except StopIteration:
            return
        history.append(next_item)
        n -= 1
    for item in sequence:
        history.append(item)
        yield tuple(history)
        del history[0]

In [60]:
with open('populate-results.json') as fopen:
    l = json.load(fopen)

In [61]:
n, p = [], []

for k, v in l.items():
    if v == 'negative':
        n.append(k)
    else:
        p.append(k)
        
len(n), len(p)

(2260, 2922)

In [62]:
negatives, shouldnot = n[:], []
for w in negate_words:
    for w_ in p:
        if w_ in ['sabar']:
            continue
        negatives.extend([f'{w}{w_}', f'{w} {w_}'])
    for w_ in n:
        shouldnot.extend([f'{w}{w_}', f'{w} {w_}'])

In [63]:
negatives_words = set(negatives)
shouldnot_words = set(shouldnot)

In [87]:
laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'zzz',
    'hoho',
    'hew',
    'uwu',
    'ooo',
    'sksk',
    'ksks',
    'gitu',
    'meow',
}

In [91]:
reject = ['sukanya', 'tolong', 'bulan', 'bau', 'pukul', 'handai', 'abis', 'visi',
'hiburkan', 'kek', 'depa', 'senyum', 'ulama', 'habis', 'bener', 'dipilih',
'suami', 'penyertaan', 'teknikal', 'jam', 'mencapai', 'kekuatan', 'hepi']

negatives_words = {w for w in negatives_words if w not in reject}

In [92]:
ns, found = [], []
for s in tqdm(cleaned):
    splitted = s[1].split()
    ngs = splitted[:]
    ngs.extend([' '.join(n) for n in ngrams(splitted, 2)])
    r = set(ngs) & negatives_words
    sn = set(ngs) & shouldnot_words
    string = [
        word
        for word in splitted
        if any([laugh in word for laugh in laughing])
    ]
    if len(r) and not len(sn) and not len(string):
        found.extend(list(r))
        ns.append(s[0])

100%|██████████| 9927217/9927217 [05:20<00:00, 31006.07it/s]


In [96]:
len(ns), len(ns) / len(cleaned)

(3103071, 0.3125821667845077)

In [97]:
with open('ns.json', 'w') as fopen:
    json.dump(ns, fopen)